In [1]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from scipy.special import expit
#import functions_hri as fnct
from functions_mendel import step, plot_results_height, plot_tanklayers_time ,plot_results_time, plot_cX_time, calculate_errors
import os
import importlib.util
import pandas as pd
from lsp_variables_mendel import z, layers, dz, d, P_i, A_i, alpha, rho, cp, k_i, beta_i, beta_top, beta_bottom, lambda_i, phi_i, dt, num_steps, T_a
from lsp_variables_mendel import t_init_interpolated_df, mdot_list_df, tm_list_df, qdot_list_df
from lsp_variables_mendel import c1_df, c2_df, c3_df, c4_df, c5_df, c6_df, c7_df, c8_df, c9_df, c10_df, e1_df, layernumber_l, connection_list, nan_indices, schichtlader_status # c8 is Schichtlader [layer, df], layernumber_l is different possible layers of outlets


# Set the working directory
working_directory = r"C:\Users\sophi\repos\repos_thesis\Model"
os.chdir(working_directory)
############### Definitons of the class for the model
class stratified_storage_tank:
 # Definition of the parameters needed to describe the heat tank and the testing conditions
    def __init__(self, alpha, beta_i, beta_bottom, beta_top, lambda_i, phi_i, z, T_a, T_initial, dt, Qdot, mdot, Tm):
        
        self.alpha = alpha                           # heat diffusivity
        
        self.beta_i = beta_i                         # heat loss coefficient to the ambient in the inner layers
        self.beta_bottom = beta_bottom               # heat loss coefficient to the ambient at the bottom layer (i=0)
        self.beta_top = beta_top                     # heat loss coefficient to the ambient at the top layer (i=-1)
        
        self.lambda_i = lambda_i                     # coefficient of the input heat
        
        self.Qdot = Qdot                             # List containing as many dfs as the max amount of qdot connections per layer. Each df contains qdot (pel) for each layer    
        
        self.phi_i = phi_i                           # coefficient of the input flow/stream
        self.mdot = mdot                             # List containing as many dfs as the max amount of stream connections per layer. Each df contains the mass rate (mdot=vdot*rho) for each layer i
        self.Tm = Tm                                 # List containing as many dfs as the max amount of stream connections per layer. Each df contains the temperatures (Tm) for each layer i
        self.z = z                                   # height of the tank
        self.num_layers = T_initial.shape[1]             # number of layers (steps in space)
        self.dz = z / self.num_layers                # step size in space (delta z, height of a layer) is total height/number of layers, num_layers = how big the initial temperature vector is
        self.heights = [i * self.dz + self.dz/2 for i in range(len(T_initial))]     # list representing the height of the tank for plotting the temperatures in the middle of each layer
        self.dt = dt                                 # step size in time (time step)
        self.T_initial = T_initial                   # df containing the interpolated tank temperatures for each layer (column) over time (indexing, rows)
        self.T_a = T_a                               # ambient temperature outside of the tank
     
 # definition of the solver to calculate the temperature of the layers (vector) in the next time step       
    def vector_solve(self, num_steps, 
                     incl_diffusivity=True, 
                     incl_heat_loss=True, 
                     incl_fast_buoyancy=True,
                     incl_slow_buoyancy=True):          # all effects are by default turned on
        
        #### Model check:
        # Check if the time step dt is a multiple of frequency
        storage_frequency = 60
        freq = int(self.dt/storage_frequency)                         # frequency of the stored values to match the time step of the simulation dt
        if self.dt % storage_frequency != 0:
            raise ValueError(f"The time step dt ({self.dt}s) must be a multiple of the data storage frequency ({freq}s).")
        
        # Check if there is enough stored data to run the desired simulation step
        num_data_available = int(len(self.T_initial) / freq)
        if num_steps > num_data_available:
            raise ValueError(f"Number of steps for the simulation ({num_steps}) must be smaller or equal to number of available stored data ({num_data_available}).")
        
        print("Thermodynamical effects of the model:")
        print("- Heat diffusivity:", incl_diffusivity) 
        print("- Heat losses:",incl_heat_loss)
        print("- Fast Buoyancy:",incl_fast_buoyancy)
        print("- Slow Buoyancy:",incl_slow_buoyancy)

        print("List of Parameters:")
        print("- Diffusivity alpha [m²/s]:",alpha)
        print("- k-Wert tank wall [W/m²K]:", k_i)
        print("- Coefficient of heat losses beta i [1/s]:", beta_i)
        print("- Coefficient of heat losses beta bottom [1/s]:", beta_bottom)
        print("- Coefficient of heat losses beta top [1/s]:", beta_top)
        print("- Coefficient of the indirect heat [mK/J]:", lambda_i)
        print("- Coefficient of the direct heat [m²/kg]:", phi_i)
        
        
     # Initialize vector for the temperature of the layers
        T_initial_arr = self.T_initial.to_numpy()[0] # self.T_initial = t_init_interpolated_df -> select first entry of array (T_initial)
        T_old = np.copy(T_initial_arr)
        results = [T_old.copy()]                    # Store initial temperature array
     # Schichtlader df
        if schichtlader_status == True:
            schichtl_df = c8_df[1]
            schichtl_arr = schichtl_df.to_numpy()
            schichtl_layer_active = np.zeros(num_steps)   # what layer is c8 entering at each time k"""
     # Iterate over k time steps
        for k in range(num_steps):
            
            # T_old is either the initial temperature (iteration 0) or the result of the new temperature of the last time step, which becomes the old temperature of the current time step (k loop)
            T_old_next = np.roll(T_old, -1)         # roll every i by -1 so that the "next" i is selected (T_k,i+1)
            T_old_prev = np.roll(T_old, 1)          # roll every i by 1 so that the "previous" i is selected (T_k,i-1)
          ###############################################################################  
          ##### CALCULATE THE INPUTS THAT WILL AFFECT THE TANK (Qdot and mdot,Tm) #######
          ###############################################################################
          # The effects caused by fast buoyancy are modeled by assuming that the physical position is replaced by fictive inputs homogenously distributed along the tank if buoyancy conditions are met.
            # Initiate matrices with layer infos (layer temperatures for time k)
            matrix_l = np.tile(T_old, (len(T_old), 1))      # create matrix with T as columns for len(T) rows (T_i const as row)
            matrix_i = matrix_l.T                           # create matrix with T as row for len(T) columns (T_l const as column)
            
            ######## INDIRECT (Qdot)
            # The amount of input in each layer is no longer the initial Qdot, but a Qdot prime that contains the collateral charge/discharge of each layer due to charging/discharging in other (and itself) layers and the temperature differences allowing buoyancy effect
            ##########################
            Qdot_prime_char_loop = np.zeros(self.num_layers)
            Qdot_prime_dischar_loop = np.zeros(self.num_layers)
            Qdot_nobuo = np.zeros(self.num_layers)
            
            # iterate over all dataframes in qdot. They allow multiple exchanger in one layer. One df contains 1 qdot per layer. Second df contains, in the layers where more than 1 qdot is connected, the data of the second qdot. Until u
            for u in range(len(self.Qdot)):
                Qdot_arr = self.Qdot[u].to_numpy()
                ####### Separate Qdot into charging and discharging vectors
                Qdot_charging = np.where(Qdot_arr[k*freq] > 0, Qdot_arr[k*freq], 0)
                Qdot_discharging = np.where(Qdot_arr[k*freq] < 0, Qdot_arr[k*freq], 0)
                Qdot_nobuo += Qdot_arr[k*freq]                                              # without buoyancy, the total amount of incoming mdot per layer is the sum of all mdots of each layer (sum over w)
                # Initiate vectors fo the buoyancy qdots
                Qdot_prime_char = np.zeros(self.num_layers)  # initiate vector with length like number of layers
                Qdot_prime_dischar = np.zeros(self.num_layers)  # initiate vector with length like number of layers
                ####
                # Indir CHARGING
                matrix_bool_qchar = np.where(matrix_l >= matrix_i, 1, 0) # check availability of heat exchange to the layer i from layer l
                # Selecting the bottom left half (diagonal) of the result_matrix (only layers where it is charged below i matter)
                matrix_nom_qchar = np.tril(matrix_bool_qchar)
                # Sum the values in each column, this is the denominator for the factor by which Ql will be multiplied
                den_sums_qchar = np.sum(matrix_nom_qchar, axis=0)  # axis 0 are columns
                
                # calculate the factor (distirbution of Ql)
                with np.errstate(divide='ignore', invalid='ignore'):
                    factor_qchar = np.where(den_sums_qchar != 0, 1 / den_sums_qchar, 0) # calculate 1/sum, prevent division by 0
                # Nom * Den for each matrix element
                matrix_factor_qchar = matrix_nom_qchar * factor_qchar
                # Multiply the factor matrix with the Qdot vector
                matrix_qchar = matrix_factor_qchar * Qdot_charging
                # The total amount of Qdot_prime for each layer is the sum of the rows of matrix_char
                Qdot_prime_char = np.sum(matrix_qchar, axis=1)   # axis 1 are rows

                Qdot_prime_char_loop += Qdot_prime_char  # Accumulate the results for the Qdot per layer (Qdot[u])
                            
                ####
                # Indir DISCHARGING
                matrix_bool_qdischar = np.where(matrix_i >= matrix_l, 1, 0) # check availability of heat exchange to the layer i from layer l
                # Selecting the top right half (diagonal) of the result_matrix (only layers above i matter)
                matrix_nom_qdischar = np.triu(matrix_bool_qdischar)
                # Sum the values in each column, this is the denominator for the factor by which Ql will be multiplied
                den_sums_qdischar = np.sum(matrix_nom_qdischar, axis=0)  # axis 0 are columns
                # calculate the facor (distirbution of Ql)
                with np.errstate(divide='ignore', invalid='ignore'):
                    factor_qdischar = np.where(den_sums_qdischar != 0, 1 / den_sums_qdischar, 0)    # calculate 1/sum, prevent division by 0
                # Nom * Den for each matrix element
                matrix_factor_qdischar = matrix_nom_qdischar * factor_qdischar
                # Multiply the factor matrix with the Qdot vector
                matrix_qdischar = matrix_factor_qdischar * Qdot_discharging
                # The total amount of Qdot_prime for each layer is the sum of the rows of matrix_char
                Qdot_prime_dischar = np.sum(matrix_qdischar, axis=1)   # axis 1 are rows
                Qdot_prime_dischar_loop += Qdot_prime_dischar
                


            ######## DIRECT (mdot,Tm), ENERGY CHANGE IS GIVEN BY mdot*cp*dT
            # The fast buoyancy effect will cause the distribution of mdot,Tm along multiple layers of the tank, provided buoyancy conditions are met.
            # the resulting mdot_prime contains the sum of the energy changes in the layers through this new separation of the inputs. 
            #############################
            if schichtlader_status == True:
                # Assign Schichtlader to the corresponding layer. Connection 8 has four possible outlets.
                schichtl_k = schichtl_arr[k*freq]                 # temp, vdot and mt of schichtlader c8 (schictl_arr) in time k
                schichtl_t_k = schichtl_k[0]                      # [0] corresponds to temperature of c8
                schichtl_mdot_k = schichtl_k[1]*(rho)#/3600000)     # [1] corresponds to vdot of c8 in m³/s
                #schichtl_mt_k = schichtl_k[2]*(rho)#/3600000)       # [2] corresponds to mt = mcpT of c8 in m³/s 
                schichtlader_layers = layernumber_l[::-1]         # layers where outlets are located, organized from smallest (bottom) to biggest (top)

                temp_schichtlayers = T_old[schichtlader_layers]   # temperature of the layers where outlets are located
                schichtlader_layer_k = schichtlader_layers[np.argmin(np.abs(temp_schichtlayers - schichtl_t_k))]  # c8 will flow to the layer where the temp difference between stream and layer is the smallest. This returns the first layer where condition is met (bottom
                
            mdot_prime_char_loop = np.zeros(self.num_layers)
            mdot_in_char_tot_loop = np.zeros(self.num_layers)
            mdot_prime_dischar_loop = np.zeros(self.num_layers)
            mdot_in_dischar_tot_loop = np.zeros(self.num_layers)
            mdot_in_tot_loop = np.zeros(self.num_layers)
            mdot_in_nobuo = np.zeros(self.num_layers)
            mdot_prime_nobuo = np.zeros(self.num_layers)
            mdot_out = np.zeros(self.num_layers)
            #test = self.mdot[1]
            # iterate over all dataframes stored in mdot. They allow multiple stream connections. One df contains 1 stream per layer. Second df contains, in the layers where more than 1 stream is connected, the data of the second stream. Until w
            for w in range(len(self.mdot)):
                #test= len(self.mdot)-1
                    
                mdot_arr = self.mdot[w].to_numpy()          # convert list element w into numpy array. This array contains the mass rate of one stream per layer.
                Tm_arr = self.Tm[w].to_numpy()              # convert list element w into numpy array. This array contains the temperature of one stream connected to each layer
                ####### Separate mdot into incoming and outgoing vectors, only incoming can charge/discharge
                mdot_in = np.where(mdot_arr[k*freq] > 0, mdot_arr[k*freq], 0)         # vector with only positive mdots
                mdot_out_w = np.where(mdot_arr[k*freq] < 0, mdot_arr[k*freq], 0)        # vector with only negative mdots
                mdot_out += mdot_out_w                                               # cummulative mdot_out vector over w dfs
                Tm_in = np.where(mdot_arr[k*freq] > 0, Tm_arr[k*freq], 0)             # vector with the temperature of the inflowing (+) streams mdot
                mdot_in_nobuo += mdot_in                                              # cumulative mdot_in without buoyancy, the total amount of incoming mdot per layer is the sum of all mdots of each layer (sum over w)
                mdot_prime_nobuo += mdot_in *(Tm_in - T_old)                            # amount of heat transferred from original incoming stream (no buo)
                if schichtlader_status == True:
                    # Assign Schichtlader data to the corresponding layer k in the last df w
                    if w == len(self.mdot)-1:
                        Tm_in[schichtlader_layer_k] = schichtl_t_k.copy()        # temp of schichtlader entering assigned layer 
                        mdot_in[schichtlader_layer_k] = schichtl_mdot_k.copy()       # mdot of schichtlader entering assigned layer, it can only flow in, no mdot_out 
                        schichtl_layer_active[k] = schichtlader_layer_k.copy()
                    
                
                matrix_Tm = np.tile(Tm_in, (len(Tm_in), 1))      # create matrix with Tm as columns for len(T) rows (Tm_l const as row)
                # differentiate between charging (Tm>Tl) and discharging (Tm<Tl) by checking the diagonal of the TmTl matrix
                mdot_in_charging = np.where(Tm_in > T_old, mdot_in, 0)
                mdot_in_discharging = np.where(Tm_in < T_old, mdot_in, 0)
                mdot_in_neutral = np.where(Tm_in == T_old, mdot_in, 0)  # if Tm - Told = 0, there is no buoyancy and the mdot enters completely this layer (without changin its temp)
                # Initiate vectors for charging and discharging
                mdot_prime_char = np.zeros(self.num_layers)  # initiate vector with length like number of layers
                mdot_prime_dischar = np.zeros(self.num_layers)  # initiate vector with length like number of layers
                mdot_in_char_tot = np.zeros(self.num_layers)    # initiate vector: the total amount of mass streaming into i through charging mdots in l will be saved here
                mdot_in_dischar_tot = np.zeros(self.num_layers)    # initiate vector: the total amount of mass streaming into i through discharging mdots in l will be saved here
                #mdot_in_tot = np.zeros(self.num_layers)            # initiate vector: total amount of mdot flowing into layer i (charge+discharge incl buoyancy)
                ####
                # dir CHARGING
                matrix_diff_mchar = matrix_Tm - matrix_i
                # matrix_diff_mchar = np.where(matrix_diff_mchar == -0, 0, matrix_diff_mchar)
                matrix_bool_mchar = np.tril(np.where(matrix_diff_mchar > 0, 1, 0)) # check availability of heat exchange to the layer i from layer l
                # Selecting the bottom left half (diagonal) of the result_matrix (only layers charged below i matter)
                matrix_nom_mchar = matrix_bool_mchar * matrix_diff_mchar # Tm - Ti (diff) as nominator for bool=1, 0 for bool=0
                matrix_nom_mchar = np.where(matrix_nom_mchar == -0.0, 0.0, matrix_nom_mchar)
                # Distribution of mdot_l: Sum the values in each column, this is the denominator for the factor by which mdotl will be multiplied
                den_sums_mchar = np.sum(matrix_bool_mchar, axis=0)  # axis 0 are columns
                # calculate the factor (distirbution of mdot_l)
                with np.errstate(divide='ignore', invalid='ignore'):
                    factor_mchar = np.where(den_sums_mchar != 0, 1 / den_sums_mchar, 0)    # calculate 1/sum, prevent division by 0
                # Nom * Den for each matrix element
                matrix_bool_factor_mchar = matrix_bool_mchar * factor_mchar
                mdot_in_char_tot = np.sum(matrix_bool_factor_mchar*mdot_in_charging, axis=1)
                mdot_in_char_tot_loop += mdot_in_char_tot # not used, just check. The sum is made in mdot_in_tot_loop further below using mdot_in_char_tot
                matrix_factor_mchar = matrix_nom_mchar * factor_mchar
                # Multiply the factor matrix with the mdot vector
                matrix_mchar = matrix_factor_mchar * mdot_in_charging
                # The total amount of mdot_prime for each layer is the sum of the rows of matrix_char
                mdot_prime_char = np.sum(matrix_mchar, axis=1)   # axis 1 are rows
                mdot_prime_char_loop += mdot_prime_char
                ####
                # dir DISCHARGING
                matrix_diff_mdischar = matrix_Tm - matrix_i
                # Selecting the top right half (diagonal) of the result_matrix (only layers above i matter)
                matrix_bool_mdischar = np.triu(np.where(matrix_i - matrix_Tm > 0, 1, 0)) # check availability of heat exchange to the layer i from layer Tm l
                matrix_nom_mdischar = matrix_diff_mdischar * matrix_bool_mdischar # temperature difference * availability of heat exchange
                matrix_nom_mdischar = np.where(matrix_nom_mdischar == -0.0, 0.0, matrix_nom_mdischar)
                # Distribution of mdot_l: Sum the values in each column, this is the denominator for the factor by which mdotl will be multiplied
                den_sums_mdischar = np.sum(matrix_bool_mdischar, axis=0)  # axis 0 are columns
                # calculate the factor (distirbution of mdot_l)
                with np.errstate(divide='ignore', invalid='ignore'):
                    factor_mdischar = np.where(den_sums_mdischar != 0, 1 / den_sums_mdischar, 0)    # calculate 1/sum, prevent division by 0
                # Nom * Den for each matrix element
                matrix_bool_factor_mdischar = matrix_bool_mdischar * factor_mdischar
                mdot_in_dischar_tot = np.sum(matrix_bool_factor_mdischar*mdot_in_discharging, axis=1)
                mdot_in_dischar_tot_loop += mdot_in_dischar_tot # not used, just check
                matrix_factor_mdischar = matrix_nom_mdischar * factor_mdischar
                # Multiply the factor matrix with the mdot vector
                matrix_mdischar = matrix_factor_mdischar * mdot_in_discharging
                # The total amount of mdot_prime for each layer is the sum of the rows of matrix_char
                mdot_prime_dischar = np.sum(matrix_mdischar, axis=1)   # axis 1 are rows
                mdot_prime_dischar_loop += mdot_prime_dischar
                mdot_in_tot_loop += mdot_in_char_tot + mdot_in_dischar_tot + mdot_in_neutral # sums the inflowing mdot to each layer (through charging, discharging and neutral) inlcuding buoyancy effects
            #mdot_in_tot = mdot_in_char_tot_loop + mdot_in_dischar_tot_loop # total amount of mdot flowing into layer i inlcuding buoyancy effects
            #mdot_in_tot = np.where(T_old == Tm_in, mdot_in, mdot_in_tot)
            
            ##### FORCED MIXING
            ###################
            # Calculate mdot mix_in for layer i. Positive means from below to above (i-1 -> i). i=0-1 is 0 (outside of tank) and i=N+1= 0 (outside of tank)
            mdot_mix_in = np.zeros(self.num_layers+1)
            
            if incl_fast_buoyancy == False:
                mdot_in_tot = mdot_in_nobuo
            else:
                mdot_in_tot = mdot_in_tot_loop
            # Calculate cumulative sum of the differences of the incoming - outgoing streams in each layer
            mdot_netto = mdot_in_tot + mdot_out                              # difference of incoming streams (also after considering buoyancy) and outgoing streams (negative in their value)
            cumulative_mdot_netto = np.cumsum(mdot_netto)                     # vector containing the cummulation of the sums of the net streams flowing out of the layers below
            mdot_mix_above = -cumulative_mdot_netto             # negative since it is "leaving" the layer
            mdot_mix_below = np.roll(cumulative_mdot_netto,+1)  # positive since it is "entering" the layer
            mdot_mix_prime_above = np.where(mdot_mix_above > 0, (mdot_mix_above * (T_old_next - T_old)), 0)
            mdot_mix_prime_below = np.where(mdot_mix_below > 0, (mdot_mix_below * (T_old_prev - T_old)), 0)
            mdot_prime_mix = mdot_mix_prime_above + mdot_mix_prime_below    # total amount of heat portion that will affect each layer by incoming and outflowing mass flows from forced streaming (direct charging/discharging)                        
            ################ Calculate new temperatures after dt
            # separate the effects of the model to allow modularity
            # initiate arrays
            
            diffusivity = np.zeros_like(T_old)
            heat_loss = np.zeros_like(T_old)
            fast_buoyancy_qdot_charge = np.zeros_like(T_old)
            fast_buoyancy_qdot_discharge = np.zeros_like(T_old)
            fast_buoyancy_mdot_charge = np.zeros_like(T_old)
            fast_buoyancy_mdot_discharge = np.zeros_like(T_old)
            slow_buoyancy  = np.zeros_like(T_old)
            # calculation of termns for the energy balance based equation
            diffusivity = ((self.alpha) * (T_old_next - (2*T_old) + T_old_prev) / (self.dz**2))     if incl_diffusivity else 0
            heat_loss = (self.beta_i * (self.T_a - T_old))                                          if incl_heat_loss else 0
            fast_buoyancy_qdot_charge = ((self.lambda_i/self.dz) * Qdot_prime_char_loop)                 if incl_fast_buoyancy else ((self.lambda_i/self.dz) * Qdot_nobuo)
            fast_buoyancy_qdot_discharge = ((self.lambda_i/self.dz) * Qdot_prime_dischar_loop)           if incl_fast_buoyancy else 0
            fast_buoyancy_mdot_charge = ((self.phi_i/self.dz) * mdot_prime_char_loop)                    if incl_fast_buoyancy else ((self.phi_i/self.dz) * mdot_prime_nobuo)
            fast_buoyancy_mdot_discharge = ((self.phi_i/self.dz) * mdot_prime_dischar_loop)              if incl_fast_buoyancy else 0 
            slow_buoyancy = ((0.5 * np.maximum(0,(T_old_prev - T_old)))                
                            - (0.5 * np.maximum(0,(T_old - T_old_next))))                                if incl_slow_buoyancy else 0
            mix_mdot_internal = ((self.phi_i/self.dz) * mdot_prime_mix)
            
            
            # Energy balance based equation
            T_new = (T_old
                     + (diffusivity
                        + heat_loss
                        + fast_buoyancy_qdot_charge
                        + fast_buoyancy_qdot_discharge
                        + fast_buoyancy_mdot_charge
                        + fast_buoyancy_mdot_discharge
                        + mix_mdot_internal
                     )* self.dt
                     + slow_buoyancy
                    ) 
            
            #### Boundary conditions
            ### bottom of the tank (i=0)
            # separate the effects of the model to allow modularity
            diffusivity_bottom = ((self.alpha) * (T_old[1] - (2*T_old[0]) + T_old[0]) / (self.dz**2))   if incl_diffusivity else 0
            heat_loss_bottom = ((self.beta_i + self.beta_bottom) * (self.T_a - T_old[0]))               if incl_heat_loss else 0
            fast_buoyancy_qdot_charge_bottom = ((self.lambda_i/self.dz) * Qdot_prime_char_loop[0])           if incl_fast_buoyancy else ((self.lambda_i/self.dz) * Qdot_nobuo[0])
            fast_buoyancy_qdot_discharge_bottom = ((self.lambda_i/self.dz) * Qdot_prime_dischar_loop[0])     if incl_fast_buoyancy else 0 
            fast_buoyancy_mdot_charge_bottom = ((self.phi_i/self.dz) * mdot_prime_char_loop[0])              if incl_fast_buoyancy else ((self.phi_i/self.dz) * mdot_prime_nobuo[0])
            fast_buoyancy_mdot_discharge_bottom = ((self.phi_i/self.dz) * mdot_prime_dischar_loop[0])        if incl_fast_buoyancy else 0
            mix_mdot_internal_bottom = ((self.phi_i/self.dz) * mdot_prime_mix[0])
            slow_buoyancy_bottom = (- 0.5 * np.maximum(0,(T_old[0] - T_old[1])))     if incl_slow_buoyancy else 0
            
            T_new[0] = (T_old[0]
                     + (diffusivity_bottom
                        + heat_loss_bottom
                        + fast_buoyancy_qdot_charge_bottom
                        + fast_buoyancy_qdot_discharge_bottom
                        + fast_buoyancy_mdot_charge_bottom
                        + fast_buoyancy_mdot_discharge_bottom
                        + mix_mdot_internal_bottom
                     )* self.dt
                     + slow_buoyancy_bottom
                    ) 
           
            ### top of the tank (i=-1)
            # separate the effects of the model to allow modularity
            diffusivity_top = ((self.alpha) * (T_old[-1] - (2*T_old[-1]) + T_old[-2]) / (self.dz**2))       if incl_diffusivity else 0
            heat_loss_top = ((self.beta_i + self.beta_top) * (self.T_a - T_old[-1]))                         if incl_heat_loss else 0
            fast_buoyancy_qdot_charge_top = ((self.lambda_i/self.dz) * Qdot_prime_char_loop[-1])                 if incl_fast_buoyancy else ((self.lambda_i/self.dz) * Qdot_nobuo[-1])
            fast_buoyancy_qdot_discharge_top = ((self.lambda_i/self.dz) * Qdot_prime_dischar_loop[-1])           if incl_fast_buoyancy else 0 
            fast_buoyancy_mdot_charge_top = ((self.phi_i/self.dz) * mdot_prime_char_loop[-1])                    if incl_fast_buoyancy else ((self.phi_i/self.dz) * mdot_prime_nobuo[-1])
            fast_buoyancy_mdot_discharge_top = ((self.phi_i/self.dz) * mdot_prime_dischar_loop[-1])              if incl_fast_buoyancy else 0
            mix_mdot_internal_top = ((self.phi_i/self.dz) * mdot_prime_mix[-1])
            slow_buoyancy_top = (0.5 * np.maximum(0,((T_old[-2] - T_old[-1]))))            if incl_slow_buoyancy else 0
            
            T_new[-1] = (T_old[-1]
                     + (diffusivity_top
                        + heat_loss_top
                        + fast_buoyancy_qdot_charge_top
                        + fast_buoyancy_qdot_discharge_top
                        + fast_buoyancy_mdot_charge_top
                        + fast_buoyancy_mdot_discharge_top
                        + mix_mdot_internal_top
                     )* self.dt
                     + slow_buoyancy_top
                    ) 

            T_old = np.copy(T_new) # return the new temperature as old temperature for the next iteration
            results.append(T_old.copy())            # Store the updated temperature array fo later plot
        return T_old, results, freq
     
 # check the stability of the model with the selected dt
    def stability_check(self):
        # check if the time step dt is small enough with CFL condition: dt <= (dz^2) / (2 * alpha)
        cfl_dt_max = (self.dz ** 2) / (2 * self.alpha)
        if self.dt > cfl_dt_max:
            print(f"Warning: Time step size dt {self.dt} exceeds CFL stability limit ({cfl_dt_max}).")
            sc = 1
        else:
            sc = 0
        return sc



# Definition of the inputs for the simulation from variables file
T_zero=t_init_interpolated_df
mdot = mdot_list_df
Tm = tm_list_df
Qdot = qdot_list_df

# define name for the model class
tank_vector5 = stratified_storage_tank(alpha, beta_i, beta_bottom, beta_top, lambda_i, phi_i, z, T_a, T_zero, dt, Qdot, mdot, Tm)
# apply stability check to the class
stability5 = tank_vector5.stability_check()
if (stability5 == 0):
    # Solve for the temperatures
    final_temperature5, results5, freq5 = tank_vector5.vector_solve(num_steps, 
                                                             incl_diffusivity=True,
                                                             incl_heat_loss=True,
                                                             incl_fast_buoyancy=True,
                                                             incl_slow_buoyancy=True)
    
    # store the timestamps
    timestamp_df = pd.DataFrame(index=T_zero.index[::freq5][:num_steps+1])
    results_df = pd.DataFrame(results5, index=T_zero.index[::freq5][:num_steps+1])
    
    # CHECK FOR NANS
    # Check if any index in timestamp_df is in nan_indices
    has_nan_indices = timestamp_df.index.isin(nan_indices).any()
    print("Are there any matching indices in nan_indices?", has_nan_indices)
    # Check if any index in timestamp_df is in nan_indices and print them
    matching_indices = timestamp_df.index[timestamp_df.index.isin(nan_indices)]
    print("Datetime indices with NaN values:")
    for index in matching_indices:
        print(index)

    # calculate and print error metrics
    error_df = calculate_errors(results_df, t_init_interpolated_df)
    print("Error metrics:", error_df)


    # Plot the results
    #plot_results_height(results5, tank_vector5.heights, dt, z, dz, "Layer temperatures over tank height. M5: integrated direct charging")
    # simulated results over time
    plot_results_time(results5, dt, timestamp_df, "Simulated temperature development of each layer over time.")
    # measured tank temperatures over time
    plot_tanklayers_time(T_zero, dt, freq5, num_steps, timestamp_df, "Original temperature development of each layer over time.")
    # measured connections over time (temperature)
    #plot_cX_time(connection_list, 2, dt, freq5, num_steps, timestamp_df, "Energy of the streams over time (mdot*cp*T)")
    #plot_cX_time(connection_list, 0, dt, freq5, num_steps, timestamp_df, "Temperature of the streams over time")
    
    #plot_cX_time(connection_list, 1, dt, freq5, num_steps, timestamp_df, "Mass rates of the streams over time")
    
    #plot_mts_time([c1_df, c2_df, c3_df, c4_df, c5_df, c6_df, c7_df, c8_df, c9_df, c10_df], 2, dt, freq5, num_steps, "mdot*cp*T")"""
    

    #plot_tanklayers_time(mt[2], dt, freq5, num_steps, "test1")
    #plot_tanklayers_time(Qdot[0], dt, freq5, num_steps, "qdot")
    #plot_mts_time([c1_df, c2_df, c3_df, c4_df, c5_df, c6_df, c7_df, c8_df, c9_df, c10_df], 1, dt, freq5, num_steps, "mdot")
    #plot_mts_time([c1_df, c2_df, c3_df, c4_df, c5_df, c6_df, c7_df, c8_df, c9_df, c10_df], 0, dt, freq5, num_steps, "T")

    #plot_tanklayers_time(T_zero, dt, freq5, num_steps, timestamp_df, "Original temperature development of each layer over time.")



Thermodynamical effects of the model:
- Heat diffusivity: True
- Heat losses: True
- Fast Buoyancy: True
- Slow Buoyancy: True
List of Parameters:
- Diffusivity alpha [m²/s]: 1.46e-07
- k-Wert tank wall [W/m²K]: 0.5
- Coefficient of heat losses beta i [1/s]: 6.060007184986919e-07
- Coefficient of heat losses beta bottom [1/s]: 6.390023593352465e-07
- Coefficient of heat losses beta top [1/s]: 6.390023593352465e-07
- Coefficient of the indirect heat [mK/J]: 4.883443537534531e-07
- Coefficient of the direct heat [m²/kg]: 0.0020442094648119545
Are there any matching indices in nan_indices? False
Datetime indices with NaN values:
Error metrics:              RMSE       MAE     MaxAE
0        2.193995  1.871717  3.937959
1        1.445309  1.352002  2.030165
2        1.273568  1.194222  2.076287
3        1.087030  0.985250  2.063727
4        0.886223  0.773050  1.893995
5        0.688788  0.600114  1.449295
6        1.196410  1.100230  1.853102
7        1.467890  1.393721  2.205828
8        

In [6]:
####### Energie im Tank WW

###  GROUP 1: WW -> Warmwasser geht aus dem tank in position c1 (für die 2 HK) und kommt zurück als c7
#    c1 = c9 + c4
gr_17_list = [c1_df[1], c7_df[1]]

gr_17 = pd.concat((df.iloc[::freq5][:num_steps+1] for df in gr_17_list), axis = 1)
gr_17["dmdot"] = gr_17["mdot_c7"] - gr_17["mdot_c1"]
gr_17["dT"] = gr_17["t_c1"] - gr_17["t_c7"]
gr_17["mdot"] = gr_17["mdot_c1"]
gr_17["q_mdot"] = gr_17["dT"] * gr_17["mdot"] *cp

# Creating the plot
fig = go.Figure()

# Adding dT line
fig.add_trace(go.Scatter(
    x=gr_17.index,
    y=gr_17["t_c1"],
    name="t_c1",
    yaxis="y1",
    line=dict(color="lightslategray", dash='dash')
))

fig.add_trace(go.Scatter(
    x=gr_17.index,
    y=gr_17["t_c7"],
    name="t_c7",
    yaxis="y1",
    line=dict(color="lightsteelblue", dash='dash')
))

fig.add_trace(go.Scatter(
    x=gr_17.index,
    y=gr_17["dT"],
    name="dT",
    yaxis="y1",
    line=dict(color="olivedrab")
))

# Adding mdot line
fig.add_trace(go.Scatter(
    x=gr_17.index,
    y=gr_17["mdot"],
    name="mdot",
    yaxis="y2",
    line=dict(color="mediumturquoise")
))

# Adding q_mdot line
fig.add_trace(go.Scatter(
    x=gr_17.index,
    y=gr_17["q_mdot"],
    name="q_mdot",
    yaxis="y2",  # Using the same y-axis as mdot for demonstration
    line=dict(color="mediumorchid")
))

# Updating layout for the dual y-axis
fig.update_layout(
    title="DHW circuit (c1=c7)",
    xaxis_title="Time",
    yaxis=dict(
        title="dT",
        titlefont=dict(color="blue"),
        tickfont=dict(color="blue"),
        anchor="x",
        side="left"
    ),
    yaxis2=dict(
        title="mdot and q_mdot",
        titlefont=dict(color="red"),
        tickfont=dict(color="red"),
        overlaying="y",
        anchor="x",
        side="right"
    )
)

# Show the plot
fig.show()




In [7]:
####### Energie im Tank HP1HG

###  GROUP 2: WP1_HG -> warmes wasser geht in c2 rein und fliesst wieder kalt im c4 ab
#    c2 = c4
#gr23 = pd.concat([c2_df[1], c3_df[1]], axis = 1)

gr_24_list = [c2_df[1], c4_df[1]]

gr_24 = pd.concat((df.iloc[::freq5][:num_steps+1] for df in gr_24_list), axis = 1)
gr_24["dmdot"] = gr_24["mdot_c2"] - gr_24["mdot_c4"]
gr_24["dT"] = gr_24["t_c2"] - gr_24["t_c4"]
gr_24["mdot"] = gr_24["mdot_c2"]
gr_24["q_mdot"] = gr_24["dT"] * gr_24["mdot"] * cp

# Creating the plot
fig = go.Figure()

# Adding dT line
fig.add_trace(go.Scatter(
    x=gr_24.index,
    y=gr_24["t_c2"],
    name="t_c2",
    yaxis="y1",
    line=dict(color="lightslategray", dash='dash')
))

fig.add_trace(go.Scatter(
    x=gr_24.index,
    y=gr_24["t_c4"],
    name="t_c4",
    yaxis="y1",
    line=dict(color="lightsteelblue", dash='dash')
))

fig.add_trace(go.Scatter(
    x=gr_24.index,
    y=gr_24["dT"],
    name="dT",
    yaxis="y1",
    line=dict(color="olivedrab")
))

# Adding mdot line
fig.add_trace(go.Scatter(
    x=gr_24.index,
    y=gr_24["mdot"],
    name="mdot",
    yaxis="y2",
    line=dict(color="mediumturquoise")
))

# Adding q_mdot line
fig.add_trace(go.Scatter(
    x=gr_24.index,
    y=gr_24["q_mdot"],
    name="q_mdot",
    yaxis="y2",  # Using the same y-axis as mdot for demonstration
    line=dict(color="mediumorchid")
))

# Updating layout for the dual y-axis
fig.update_layout(
    title="HP HG circuit (c2=c4)",
    xaxis_title="Time",
    yaxis=dict(
        title="dT",
        titlefont=dict(color="blue"),
        tickfont=dict(color="blue"),
        anchor="x",
        side="left"
    ),
    yaxis2=dict(
        title="mdot and q_mdot",
        titlefont=dict(color="red"),
        tickfont=dict(color="red"),
        overlaying="y",
        anchor="x",
        side="right"
    ),
    legend=dict(x=0.1, y=0.9)
)

# Show the plot
fig.show()




In [8]:
####### Energie im Tank Boiler

###  GROUP 3: B -> warmes wasser geht in c3 rein und fliesst in c5 raus

#    c3 = c5
gr_35_list = [c3_df[1], c5_df[1]]

gr_35 = pd.concat((df.iloc[::freq5][:num_steps+1] for df in gr_35_list), axis = 1)
gr_35["mdot"] = gr_35["mdot_c3"] - gr_35["mdot_c5"]
gr_35["dT"] = gr_35["t_c3"] - gr_35["t_c5"]
gr_35["mdot"] = gr_35["mdot_c3"]
gr_35["q_mdot"] = gr_35["dT"] * gr_35["mdot"] *cp


# Creating the plot
fig = go.Figure()

# Adding dT line
fig.add_trace(go.Scatter(
    x=gr_35.index,
    y=gr_35["t_c3"],
    name="t_c3",
    yaxis="y1",
    line=dict(color="lightslategray", dash='dash')
))

fig.add_trace(go.Scatter(
    x=gr_35.index,
    y=gr_35["t_c5"],
    name="t_c5",
    yaxis="y1",
    line=dict(color="lightsteelblue", dash='dash')
))

fig.add_trace(go.Scatter(
    x=gr_35.index,
    y=gr_35["dT"],
    name="dT",
    yaxis="y1",
    line=dict(color="olivedrab")
))

# Adding mdot line
fig.add_trace(go.Scatter(
    x=gr_35.index,
    y=gr_35["mdot_c3"],
    name="mdot_c3",
    yaxis="y2",
    line=dict(color="mediumturquoise")
))

fig.add_trace(go.Scatter(
    x=gr_35.index,
    y=gr_35["mdot_c5"],
    name="mdot_c5",
    yaxis="y2",
    line=dict(color="mediumturquoise")
))


# Adding q_mdot line
fig.add_trace(go.Scatter(
    x=gr_35.index,
    y=gr_35["q_mdot"],
    name="q_mdot",
    yaxis="y2",  # Using the same y-axis as mdot for demonstration
    line=dict(color="mediumorchid")
))

# Updating layout for the dual y-axis
fig.update_layout(
    title="Boiler circuit (c3=c5)",
    xaxis_title="Time",
    yaxis=dict(
        title="dT",
        titlefont=dict(color="blue"),
        tickfont=dict(color="blue"),
        anchor="x",
        side="left"
    ),
    yaxis2=dict(
        title="mdot and q_mdot",
        titlefont=dict(color="red"),
        tickfont=dict(color="red"),
        overlaying="y",
        anchor="x",
        side="right"
    )
)

# Show the plot
fig.show()




In [9]:
####### Energie im Tank Loading

###  GROUP 4: Loading -> energy shared with solar tank 

#    c6 = c9
gr_69_list = [c6_df[1], c9_df[1]]

gr_69 = pd.concat((df.iloc[::freq5][:num_steps+1] for df in gr_69_list), axis = 1)
gr_69["dmdot"] = gr_69["mdot_c6"] - gr_69["mdot_c9"]
gr_69["dT"] = gr_69["t_c6"] - gr_69["t_c9"]
gr_69["mdot"] = gr_69["mdot_c6"]
gr_69["q_mdot"] = gr_69["dT"] * gr_69["mdot"] * cp

# Creating the plot
fig = go.Figure()

# Adding dT line
fig.add_trace(go.Scatter(
    x=gr_69.index,
    y=gr_69["t_c6"],
    name="t_c6",
    yaxis="y1",
    line=dict(color="lightslategray", dash='dash')
))

fig.add_trace(go.Scatter(
    x=gr_69.index,
    y=gr_69["t_c9"],
    name="t_c9",
    yaxis="y1",
    line=dict(color="lightsteelblue", dash='dash')
))

fig.add_trace(go.Scatter(
    x=gr_69.index,
    y=gr_69["dT"],
    name="dT",
    yaxis="y1",
    line=dict(color="olivedrab")
))

# Adding mdot line
fig.add_trace(go.Scatter(
    x=gr_69.index,
    y=gr_69["mdot"],
    name="mdot",
    yaxis="y2",
    line=dict(color="mediumturquoise")
))

# Adding q_mdot line
fig.add_trace(go.Scatter(
    x=gr_69.index,
    y=gr_69["q_mdot"],
    name="q_mdot",
    yaxis="y2",  # Using the same y-axis as mdot for demonstration
    line=dict(color="mediumorchid")
))

# Updating layout for the dual y-axis
fig.update_layout(
    title="Loanding between tanks (c6 = c9)",
    xaxis_title="Time",
    yaxis=dict(
        title="dT",
        titlefont=dict(color="blue"),
        tickfont=dict(color="blue"),
        anchor="x",
        side="left"
    ),
    yaxis2=dict(
        title="mdot and q_mdot",
        titlefont=dict(color="red"),
        tickfont=dict(color="red"),
        overlaying="y",
        anchor="x",
        side="right"
    ),
    legend=dict(x=0.1, y=0.9)
)

# Show the plot
fig.show()




In [4]:
####### Energie im Tank HP normal

###  GROUP 4: HP -> energy form normal temperature of heat pump flows into c8 and returns as c10 

#    c8 = c10
gr_810_list = [c8_df[1], c10_df[1]]

gr_810 = pd.concat((df.iloc[::freq5][:num_steps+1] for df in gr_810_list), axis = 1)
gr_810["dmdot"] = gr_810["mdot_c8"] - gr_810["mdot_c10"]
gr_810["dT"] = gr_810["t_c8"] - gr_810["t_c10"]
gr_810["mdot"] = gr_810["mdot_c8"]
gr_810["q_mdot"] = gr_810["dT"] * gr_810["mdot"] *cp

# Creating the plot
fig = go.Figure()

# Adding dT line
fig.add_trace(go.Scatter(
    x=gr_810.index,
    y=gr_810["t_c8"],
    name="t_c8",
    yaxis="y1",
    line=dict(color="lightslategray", dash='dash')
))

fig.add_trace(go.Scatter(
    x=gr_810.index,
    y=gr_810["t_c10"],
    name="t_c10",
    yaxis="y1",
    line=dict(color="lightsteelblue", dash='dash')
))

fig.add_trace(go.Scatter(
    x=gr_810.index,
    y=gr_810["dT"],
    name="dT",
    yaxis="y1",
    line=dict(color="olivedrab")
))

# Adding mdot line
fig.add_trace(go.Scatter(
    x=gr_810.index,
    y=gr_810["mdot"],
    name="mdot",
    yaxis="y2",
    line=dict(color="mediumturquoise")
))

# Adding q_mdot line
fig.add_trace(go.Scatter(
    x=gr_810.index,
    y=gr_810["q_mdot"],
    name="q_mdot",
    yaxis="y2",  # Using the same y-axis as mdot for demonstration
    line=dict(color="mediumorchid")
))

# Updating layout for the dual y-axis
fig.update_layout(
    title="HP lower temp (c8 = c10)",
    xaxis_title="Time",
    yaxis=dict(
        title="dT",
        titlefont=dict(color="blue"),
        tickfont=dict(color="blue"),
        anchor="x",
        side="left"
    ),
    yaxis2=dict(
        title="mdot and q_mdot",
        titlefont=dict(color="red"),
        tickfont=dict(color="red"),
        overlaying="y",
        anchor="x",
        side="right"
    ),
    legend=dict(x=0.1, y=0.9)
)

# Show the plot
fig.show()




In [5]:
# all qdots

# List of DataFrames and their names
dfs = [gr_17, gr_24, gr_35, gr_69, gr_810]

names = ["Group HK", "Group HPHG", "Group Boiler", "Group Loading", "Group HP"]
colors = ["blue", "grey", "red", "green", "black"]

# Create a Plotly figure
fig = go.Figure()

# Add traces for each DataFrame
for df, name, color in zip(dfs, names, colors):
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df["q_mdot"],
        name=name,
        line=dict(color=color)
    ))

# Adding q_mdot total line
fig.add_trace(go.Scatter(
    x=gr_810.index,
    y=gr_17["q_mdot"]+gr_24["q_mdot"]+gr_35["q_mdot"]+gr_69["q_mdot"]+gr_810["q_mdot"],
    name="q_mdot",
    #yaxis="y2",  # Using the same y-axis as vdot for demonstration
    line=dict(color="magenta")
))

# Update layout
fig.update_layout(
    title="q_mdot for Multiple Groups",
    xaxis_title="Index",
    yaxis_title="q_mdot",
    yaxis=dict(
        title="q_mdot",
        titlefont=dict(color="black"),
        tickfont=dict(color="black")
    ),
    #legend=dict(x=0.1, y=0.9)
)

# Show the plot
fig.show()






# all mdots

# List of DataFrames and their names
dfs = [gr_17, gr_24, gr_35, gr_69]#, gr_810]
names = ["Group HK", "Group HPHG", "Group Boiler", "Group Loading", "Group HP"]
colors = ["blue", "grey", "red", "green", "black"]

# Create a Plotly figure
fig = go.Figure()

# Add traces for each DataFrame
for df, name, color in zip(dfs, names, colors):
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df["mdot"],
        name=name,
        line=dict(color=color)
    ))


# Update layout
fig.update_layout(
    
    title="mdot for Multiple Groups",
    xaxis_title="Index",
    yaxis_title="mdot",
    yaxis=dict(
        title="mdot",
        titlefont=dict(color="black"),
        tickfont=dict(color="black")
    ),
    #legend=dict(x=0.1, y=0.9)
)

# Show the plot
fig.show()

NameError: name 'gr_35' is not defined

In [8]:
import numpy as np
import pandas as pd
#from Buildings.mendel.model.functions_mendel import create_zero_tuple, 
import os
y = os.getcwd()
############### DEFINE TANK
# Solvis Stratos 917l

# tank description
z = 1.873                               # height of the tank [m]
layers = 10
dz = z / layers                # height of the section (layer)
d = 0.79                             # diameter of the cross section [m]
P_i = np.pi * d                     # cross-sectional perimeter [m]
A_i = np.pi * (d/2)**2              # cross-sectional area [m²]
# material properties
alpha = 0.000000146                 # heat diffusivity of the fluid [m²/s]
rho = 998                           # density of the fluid [kg/m³]
cp =  4186                          # heat capacity of the fluid [J/kgK]
k_i = 0.5                           # thermal condunctance of the wall [W/m²K]
# losses: Beta calculations
beta_i = (P_i*k_i)/(rho * cp * A_i) # coefficient of heat losses through the wall of the tank [1/s]
beta_top = (k_i/(rho * cp * dz))    # coefficient of heat losses through the ceiling of the tank [1/s]
beta_bottom = (k_i/(rho * cp * dz)) # coefficient of heat losses through the ground of the tank [1/s]
# indirect heat 
lambda_i = (1/(A_i*rho*cp))         # coefficient of the indirect heat (heat echanger)
# direct heat
phi_i = (1/(A_i*rho))               # coefficient of the direct heat (mass stream)


# Heights of entries
heights_positions_tank = [1.843, 
                          1.443, 
                          1.343, 
                          1.243, 
                          1.143, 
                          1.043, 
                          0.728, 
                          0.628, 
                          0.278, 
                          0.036, 
                          0.036]  # all positions (1-12) except position 10 (schichtlader)
heights_schichtlader = [1.419, 
                        1.112, 
                        0.825, 
                        0.528]                                                     # position of the openings of the Schichtlader
heights_sensors = [1.643, 
                   1.343,
                   1.043, 
                   0.278, 
                   0.153]                                            # ** position of the 5 installed sensors in LSP Mendel

# assign the height of each connection c (9 connections)
c_z = [heights_positions_tank[0],   #c1
       heights_positions_tank[1],   #c2
       heights_positions_tank[2],   #c3
       heights_positions_tank[5],   #c4
       heights_positions_tank[6],   #c5
       heights_positions_tank[7],   #c6
       heights_positions_tank[8],   #c7
       heights_positions_tank[9],   #c9
       heights_positions_tank[10]]  #c10

l_z = [heights_schichtlader] # #c8 (1 Schichtlader)
# assign the height of the heat exchangers (1 exchanger)
e_z = [0] # no heat exchangers


# Assign the fictive layers to the physical entries
layernumber_c =  (np.array(c_z) / dz).astype(int)   # connections (streams)
layernumber_l = (np.array(heights_schichtlader) / dz).astype(int)    # layerloader (Schichtlader)
layernumber_e = (np.array(e_z) / dz).astype(int)    # heat exchanger
layernumber_sp = (np.array(heights_sensors) / dz)    # temperature sensors as float to be able to determine which sensor to use for layers with multiple sensors

##################################### IMPORTING THE DATA
# Each connector data is saved into a separate .csv file with the name cX (X=#streams). Columns = timestamp, t_cX, mdot_cX, mt_cX. mt_cX is not relevant.


###################################### TANK TEMPERATURE SENSORS
############### IMPORT DATA
#os.chdir(r'C:\Users\sophi\repos\repos_thesis\Buildings\seeweg\connections_seeweg')
path = r'C:\Users\sophi\repos\repos_thesis'
#### TEMPERATURE SENSORS
t_sp_tot = pd.read_csv(f"{path}\\Buildings\\mendel\\connections_mendel_lsp\\sp.csv",    
                 parse_dates=['timestamp'],
                 index_col='timestamp') 

t_sp1 = [layernumber_sp[0], t_sp_tot["t_lsp1"]] # pos1 TLSPos

t_sp2 = [layernumber_sp[1], t_sp_tot["t_lsp2"]] # pos4 TLSPo

t_sp3 = [layernumber_sp[2], t_sp_tot["t_lsp3"]] # pos6 TLSPm

t_sp4 = [layernumber_sp[3], t_sp_tot["t_lsp4"]] # pos9 

t_sp5 = [layernumber_sp[4], t_sp_tot["t_lsp5"]] # pos11 TLSPu

t_sp = [t_sp1, t_sp2, t_sp3, t_sp4, t_sp5]




################# TANK SENSORS ASSIGNATION TO LAYER: 
##### Assing t_init by checking all sensor positions and the defined layers. Select the sensor closest to the middle of the layer or NaN if no sensor is in the layer
# Initialize t_init list
t_init = []   # contains list with layer and sensor info [layer, df]
t_init_sensordf = [] # contains only the sensor data [df]


# Iterate through each layer index from 0 to layers-1
for n in range(layers):
    # Initialize variables to track the closest sensor
    closest_diff = np.inf
    closest_t_sp = [np.nan, None]  # Initialize with NaN values
    
    # Iterate through each t_spX in t_sp
    for t_spX in t_sp:
        # Get the layer number associated with t_spX and its difference from n + 0.5
        layernum = t_spX[0]
        diff = np.abs(layernum - (n + 0.5))
        
        # Check if this t_spX can be used for the current layer and has a smaller diff
        if diff <= 0.5 and diff < closest_diff:
            closest_diff = diff
            closest_t_sp = t_spX
    
    # Append the closest found sensor to t_init
    t_init.append(closest_t_sp)
    t_init_sensordf.append(closest_t_sp[1])

# Now t_init contains the selected temperature sensor data (closest to n + 0.5) for each layer
# Each entry in t_init will be a list like [layer number, temperature data] or [np.nan, None] if no sensor was found



###### calculate linear interpolation if layer has no sensor
# Initialize t_init list
t_init_interpolated = []

# Iterate through each layer index from 0 to layers-1
for n in range(layers):
    # Check if t_init has a valid entry or needs interpolation
    if t_init[n][1] is not None:
        # Use the existing temperature data
        t_init_interpolated.append(t_init[n][1])
    else:
        # Find the nearest layers with valid temperature data
        layer_below = None
        layer_above = None
        
        # Find the nearest layers with valid temperature data
        for layer in t_init[:n][::-1]:  # Search backwards for layer_below
            if layer[1] is not None:
                layer_below = layer
                break
        
        for layer in t_init[n+1:]:  # Search forwards for layer_above
            if layer[1] is not None:
                layer_above = layer
                break
        
        # Perform interpolation or duplicate closest valid value
        if n == 0 and layer_above is not None:
            # Duplicate closest value from layer_above
            t_init_interpolated.append(layer_above[1])
        elif n == layers - 1 and layer_below is not None:
            # Duplicate closest value from layer_below
            t_init_interpolated.append(layer_below[1])
        elif layer_below is not None and layer_above is not None:
            # Perform linear interpolation
            layernum_below = layer_below[0]
            layernum_above = layer_above[0]
            temp_below = layer_below[1]
            temp_above = layer_above[1]
            
            # Calculate interpolated temperature
            interpolated_temp = temp_below + ((n + 0.5 - layernum_below) / (layernum_above - layernum_below)) * (temp_above - temp_below)
            
            # Append the interpolated temperature to t_init_interpolated
            #t_init_interpolated.append([n + 0.5, interpolated_temp])
            t_init_interpolated.append(interpolated_temp)
        else:
            # If no valid layers found for interpolation or duplication, append NaN values
            print(f"Sensor data for layer {n} is NaN after interpolation. Check!")
            #t_init_interpolated.append([np.nan, None])  # Replace None with np.nan if needed for temperature data
            t_init_interpolated.append([None])  # Replace None with np.nan if needed for temperature data


t_init_interpolated_df = pd.concat(t_init_interpolated, axis=1)
#t_init_interpolated_arr = t_init_interpolated_df.to_numpy()

# Now t_init_interpolated contains the interpolated or duplicated temperature data for each layer
# Each entry in t_init_interpolated will be a list like [layer number, interpolated/duplicated temperature] (or [np.nan, None] if no interpolation/duplication was possible)

#t_init_interpolated1 = t_init_interpolated[][1]








###################################### CONNECTION DATA (MDOT; QDOT) ASSIGNATION TO LAYER
############### IMPORT DATA
#### CONNECTION DATA cX as [position of sensor, df with sensor data]
c1 = [layernumber_c[0], pd.read_csv(f"{path}\\Buildings\\mendel\\connections_mendel_lsp\\c1.csv",    #WW VL
                 parse_dates=['timestamp'],
                 index_col='timestamp')]
#c1[1].iloc[:,1] = c1[1].iloc[:,1]*10
c2 = [layernumber_c[1], pd.read_csv(f"{path}\\Buildings\\mendel\\connections_mendel_lsp\\c2.csv",    
                 parse_dates=['timestamp'],
                 index_col='timestamp')]

c3 = [layernumber_c[2], pd.read_csv(f"{path}\\Buildings\\mendel\\connections_mendel_lsp\\c3.csv",    
                 parse_dates=['timestamp'],
                 index_col='timestamp')]

c4 = [layernumber_c[3], pd.read_csv(f"{path}\\Buildings\\mendel\\connections_mendel_lsp\\c4.csv",  
                 parse_dates=['timestamp'],
                 index_col='timestamp')]

c5 = [layernumber_c[4], pd.read_csv(f"{path}\\Buildings\\mendel\\connections_mendel_lsp\\c5.csv",    
                 parse_dates=['timestamp'],
                 index_col='timestamp')]

c6 = [layernumber_c[5], pd.read_csv(f"{path}\\Buildings\\mendel\\connections_mendel_lsp\\c6.csv",    
                 parse_dates=['timestamp'],
                 index_col='timestamp')]

c7 = [layernumber_c[6], pd.read_csv(f"{path}\\Buildings\\mendel\\connections_mendel_lsp\\c7.csv",    
                 parse_dates=['timestamp'],
                 index_col='timestamp')]

c8 = [layernumber_l, pd.read_csv(f"{path}\\Buildings\\mendel\\connections_mendel_lsp\\c8.csv",    # schichtlader!
                 parse_dates=['timestamp'],
                 index_col='timestamp')]

c9 = [layernumber_c[7], pd.read_csv(f"{path}\\Buildings\\mendel\\connections_mendel_lsp\\c9.csv",    # WW RL2
                 parse_dates=['timestamp'],
                 index_col='timestamp')]
#c9[1].iloc[:,1] = c9[1].iloc[:,1]*10

c10 = [layernumber_c[8], pd.read_csv(f"{path}\\Buildings\\mendel\\connections_mendel_lsp\\c10.csv",    
                 parse_dates=['timestamp'],
                 index_col='timestamp')]

if e_z[0] == 0:
    e1 = [0, t_sp1[1].copy()]
    e1[1].loc[:] = 0
    e1[1].rename("pel", inplace=True)
else:
    e1 = [layernumber_e, pd.read_csv(f"{path}\\Buildings\\seeweg\\connections_seeweg\\e1.csv",    
                 parse_dates=['timestamp'],
                 index_col='timestamp')]

#### STREAM CONNECTIONS MATRIX
# Create a vector that contains the max amount of streams that are possible per layer (including Schichtlader)
layer_c_counts_vector = np.zeros(layers, dtype=int)
counts_c_normal = np.bincount(layernumber_c, minlength=layers)
counts_schichtlader = np.bincount(layernumber_l, minlength=layers)
layer_c_counts_vector[:len(counts_c_normal)] = counts_c_normal + counts_schichtlader
# Create an integer that represents the max amount of streams that appear in any layer of the tank

layer_c_counts_max = max(layer_c_counts_vector).astype(int)

# list of all "normal" connections (stream flows where height is defined)
mdot_normal = [c1, c2, c3, c4, c5, c6, c7, c9, c10]

# Initialize mdot_list as a list of lists
mdot_list = [[] for _ in range(layers)]
# Copy the DataFrame and fill all columns with 0
df_c_zero_filled = c1[1].copy()
df_c_zero_filled.loc[:, :] = 0
df_c_zero_filled.rename(columns={"t_c1": "t", "mdot_c1": "v", "mt_c1": "mt"}, inplace=True)

# Iterate through each layer to create a list mdot_list with all connected stream data (!! Schichtlader not included, this has to be evaluated on each time step in the simulation)
#        For the Schichtlader, overwrite the last entry of mdot_list (should be empty) for the corresponding layer where stream flows out
for n in range(layers):
    count = 0  # Initialize count of dataframes added to mdot_list[n]
    
    # Iterate through each normal mdot connection
    for cX in mdot_normal:
        if cX[0] == n:                      # cX[0] is layer number where cX is located. If cX is located in current layer n:
            mdot_list[n].append(cX[1])      # store t, mdot and mt of cX (cX[1])
            count += 1
    
    # Fill mdot_list[n] with df_zero_filled until layer_counts_max is achieved
    while count < layer_c_counts_max:
        mdot_list[n].append(df_c_zero_filled.copy())
        count += 1
        


# Create a list with count_c_max dataframes which contain the layers data (t_layer and vdot_layer as columns) for the intergation in the model
# Initialize tm and vdot as empty lists of lists
tm = [pd.DataFrame() for _ in range(layer_c_counts_max)]
mdot = [pd.DataFrame() for _ in range(layer_c_counts_max)]
mt = [pd.DataFrame() for _ in range(layer_c_counts_max)]

for m in range(layer_c_counts_max):
    for n in range(layers):
        tm[m] = pd.concat([tm[m], mdot_list[n][m].iloc[:,0]], axis=1)
        mdot[m] = pd.concat([mdot[m], mdot_list[n][m].iloc[:,1]*(rho)], axis=1)   # vdot in [kg/s] (/(3600*1000 already in cX .csv file
        mt[m] = pd.concat([mt[m], mdot_list[n][m].iloc[:,2]*(rho)], axis=1) 

tm_list_df = tm
mdot_list_df = mdot
mt_list_df = mt










#### EXCHANGERS CONNECTIONS MATRIX
# Create a vector that contains the max amount of exchangers that are possible per layer
layer_e_counts_vector = np.zeros(layers, dtype=int)
counts_e = np.bincount(layernumber_e, minlength=layers)
layer_e_counts_vector[:len(counts_e)] = counts_e
# Create an integer that represents the max amount of streams that appear in any layer of the tank

layer_e_counts_max = max(layer_e_counts_vector).astype(int)

# list of all "normal" connections (stream flows where height is defined)


qdot_normal = [e1]

# Initialize qdot_list as a list of lists
qdot_list = [[] for _ in range(layers)]
# Copy the DataFrame and fill all columns with 0
df_e_zero_filled = e1[1].copy()
df_e_zero_filled.loc[:] = 0
df_e_zero_filled.rename("pel", inplace=True)

# Iterate through each layer to create a list mdot_list with all connected stream data (!! Schichtlader not included, this has to be evaluated on each time step in the simulation)
#        For the Schichtlader, overwrite the last entry of mdot_list (should be empty) for the corresponding layer where stream flows out
for n in range(layers):
    count = 0  # Initialize count of dataframes added to mdot_list[n]
    
    # Iterate through each normal mdot connection
    for eX in qdot_normal:
        if eX[0] == n:
            qdot_list[n].append(eX[1])
            count += 1
    
    # Fill mdot_list[n] with df_zero_filled until layer_counts_max is achieved
    while count < layer_e_counts_max:
        qdot_list[n].append(df_e_zero_filled.copy())
        count += 1

# Create a list with count_e_max dataframes which contain the layers data (pel_layer as column) for the intergation in the model
# Initialize tm and vdot as empty lists of lists
qdot = [pd.DataFrame() for _ in range(layer_e_counts_max)]

for m in range(layer_e_counts_max):
    for n in range(layers):
        qdot[m] = pd.concat([qdot[m], qdot_list[n][m].iloc[:]], axis=1)


qdot_list_df = qdot

##############################################
dt = 1*60                             # length of the time steps [s]
#freq = int(dt/60)                         # frequency of the stored values to match the time step of the simulation dt
num_steps = 30                   # number of time steps to be made
#select rows of the data
start = 25
end = start + 25000

#timestamp_df = pd.DataFrame(index=.index[::freq5][:num_steps+1]))

T_a = 20                       # ambient temperature

############## pass only filtered dataframes with start and end
t_init_interpolated_df = t_init_interpolated_df.iloc[start:end]     # layer temperatures (sensor or interpolated) over time
mdot_list_df = [dff.iloc[start:end] for dff in mdot_list_df ]       # list of mdots with max one stream per layer

tm_list_df = [dff.iloc[start:end] for dff in tm_list_df ]           # list of temperatures of the streams with max one stream per layer
mt_list_df = [dff.iloc[start:end] for dff in mt_list_df ]           # list of mcpT with max one stream per layer
#mt_all_streams = pd.concat([df.drop(columns=['mt']) for df in mt_list_df], axis=1)
qdot_list_df = [dff.iloc[start:end] for dff in qdot_list_df ]       # list of heat connection with max one connection per layer

c1_df = c1.copy()                       #c1
c1_df[1] = c1_df[1].iloc[start:end]
#c1_df[1].iloc[:,1] = c1_df[1].iloc[:,1]*10
c2_df = c2.copy()                       #c2
c2_df[1] = c2_df[1].iloc[start:end]
c3_df = c3.copy()                       #c3
c3_df[1] = c3_df[1].iloc[start:end]
c4_df = c4.copy()                       #c4
c4_df[1] = c4_df[1].iloc[start:end]
#c4_df[1].iloc[:,1] = c4_df[1].iloc[:,1]*10
c5_df = c5.copy()                       #c5
c5_df[1] = c5_df[1].iloc[start:end]
c6_df = c6.copy()                       #c6
c6_df[1] = c6_df[1].iloc[start:end]
c7_df = c7.copy()                       #c7
c7_df[1] = c7_df[1].iloc[start:end]
c8_df = c8.copy()                       #c8
c8_df[1] = c8_df[1].iloc[start:end]
c9_df = c9.copy()                       #c9
c9_df[1] = c9_df[1].iloc[start:end]
#c9_df[1].iloc[:,1] = c9_df[1].iloc[:,1]*10
c10_df = c10.copy()                     #c10
c10_df[1] = c10_df[1].iloc[start:end]
e1_df = e1.copy()                       #e1
e1_df[1] = e1_df[1].iloc[start:end]

connection_list = [c1_df, c2_df, c3_df, c4_df, c5_df, c6_df, c7_df, c8_df, c9_df, c10_df]








